In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import pickle
import os

In [2]:
flight_data = (
    pd.read_csv('final_dataset_cleaned_v1.csv')
)

In [3]:
flight_data.shape

(12658696, 29)

In [4]:
flight_data.head(n = 3)

YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK     FL_DATE TAIL_NUM  \
0  2020        1      1             1            3  2020-01-01   N951WN   
1  2020        1      1             1            3  2020-01-01   N467WN   
2  2020        1      1             1            3  2020-01-01   N7885A   

  ORIGIN ORIGIN_CITY_NAME DEST  ...                 time  temp  dwpt  rhum  \
0    ONT      Ontario, CA  SFO  ...  2020-01-01 18:00:00  12.2  -7.3  25.0   
1    ONT      Ontario, CA  SFO  ...  2020-01-01 11:00:00   3.9  -3.4  59.0   
2    ONT      Ontario, CA  SJC  ...  2020-01-01 20:00:00  12.8  -7.3  24.0   

   prcp   wdir  wspd    pres coco target  
0   0.0  330.0  18.4  1017.3  3.0      1  
1   0.0  210.0   5.4  1017.6  2.0      0  
2   0.0  290.0  11.2  1015.8  2.0      0  

[3 rows x 29 columns]

In [5]:
flight_data['aircraft_type_new'] = [i.split('-')[0] for i in flight_data['Aircraft type']]

In [6]:
# list of numerical and categorical features to select 
l_numeric_cols = [ 'DEP_HOUR', 'temp', 'dwpt', 'rhum', 'prcp', 'wspd', 'pres', 'coco']
l_categorical_cols = ['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'aircraft_type_new']
target = "target"

In [7]:
# converting the categorical columns to categorical
for col in l_categorical_cols:
    flight_data[col] = flight_data[col].astype('category')

In [8]:
flight_data = flight_data[l_numeric_cols + l_categorical_cols + [target] + ['YEAR']]

In [9]:
# getting dummu values for categorical columns
QUARTER_dummies = pd.get_dummies(flight_data['QUARTER'], prefix='QUARTER', drop_first=True)
MONTH_dummies = pd.get_dummies(flight_data['MONTH'], prefix='MONTH', drop_first=True)
DAY_OF_MONTH_dummies = pd.get_dummies(flight_data['DAY_OF_MONTH'], prefix='DAY_OF_MONTH', drop_first=True)
DAY_OF_WEEK_dummies = pd.get_dummies(flight_data['DAY_OF_WEEK'], prefix='DAY_OF_WEEK', drop_first=True)
Aircraft_type_dummies = pd.get_dummies(flight_data['aircraft_type_new'], prefix='Aircraft_type', drop_first=True)

In [10]:
# Aircraft_type_dummies.columns = ['Aircraft_{}'.format(i) for i in range(len(Aircraft_type_dummies.columns))]

In [11]:
flight_data = flight_data.drop(l_categorical_cols, axis=1)

In [12]:
dummy_flight_data = pd.concat([flight_data, QUARTER_dummies, MONTH_dummies,
                               DAY_OF_MONTH_dummies, DAY_OF_WEEK_dummies, Aircraft_type_dummies ], axis=1)

In [13]:
del Aircraft_type_dummies,QUARTER_dummies,MONTH_dummies,DAY_OF_MONTH_dummies,DAY_OF_WEEK_dummies

In [14]:
train = dummy_flight_data[dummy_flight_data['YEAR'] == 2019]
test = dummy_flight_data[dummy_flight_data['YEAR'] == 2020]

In [15]:
del flight_data, dummy_flight_data

In [16]:
logistic_regression = LogisticRegression(multi_class="ovr")
logistic_regression.fit(
    train.drop(["YEAR","target"], axis=1).values,
     train[target]
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

LogisticRegression(multi_class='ovr')

In [19]:
path = "./models"
filename = "2019train_LR_ovr.mdl"

In [19]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(logistic_regression, f, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
# predicting the test results 
y_pred = logistic_regression.predict(test.drop(["YEAR","target"], axis=1).values)

In [21]:
print(confusion_matrix(test[target],y_pred))

[[4304383       0       0       0]
 [ 277058       0       0       0]
 [  85466       0       0       0]
 [  54326       0       0       0]]


In [22]:
print(classification_report(test[target],y_pred))

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.91      1.00      0.95   4304383
           1       0.00      0.00      0.00    277058
           2       0.00      0.00      0.00     85466
           3       0.00      0.00      0.00     54326

    accuracy                           0.91   4721233
   macro avg       0.23      0.25      0.24   4721233
weighted avg       0.83      0.91      0.87   4721233



C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
print("Testing  set score for model: %f" % logistic_regression.score(test.drop(["YEAR","target"], axis=1).values  , test[target] ))

Testing  set score for model: 0.911707


In [16]:
###### logistic regression multinomial ####
logistic_regression_mul = LogisticRegression(multi_class="multinomial")
logistic_regression_mul.fit(
    train.drop(["YEAR","target"], axis=1).values,
     train[target]
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(multi_class='multinomial')

In [20]:
filename = "2019train_LR_mul.mdl"

In [21]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(logistic_regression_mul, f, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
# predicting the test results 
y_pred = logistic_regression_mul.predict(test.drop(["YEAR","target"], axis=1).values)

In [23]:
print(confusion_matrix(test[target],y_pred))

[[4304383       0       0       0]
 [ 277058       0       0       0]
 [  85466       0       0       0]
 [  54326       0       0       0]]


In [24]:
print(classification_report(test[target],y_pred))

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.91      1.00      0.95   4304383
           1       0.00      0.00      0.00    277058
           2       0.00      0.00      0.00     85466
           3       0.00      0.00      0.00     54326

    accuracy                           0.91   4721233
   macro avg       0.23      0.25      0.24   4721233
weighted avg       0.83      0.91      0.87   4721233



C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
print("Testing  set score for model: %f" % logistic_regression_mul.score(test.drop(["YEAR","target"], axis=1).values  , test[target] ))

Testing  set score for model: 0.911707
